In [1]:
from telegram.ext import Updater, CommandHandler
import requests
import re
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.chrome.options import options
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import requests
import smtplib
import pandas as pd
import time
import re
from datetime import datetime,date

In [ ]:
def get_url():
    contents = requests.get('https://random.dog/woof.json').json()    
    url = contents['url']
    return url
def busca_zoom(x):
    #SELENIUM PARA FAZER A BUSCA  o produto atualizar quantidade de produtos por pagina E RETORNAR URL
    driver = webdriver.Chrome()
    driver.get('https://www.zoom.com.br/')
    time.sleep(2)
    driver.find_element_by_class_name("zsearch__fake").click()
    time.sleep(1)
    search = driver.find_element_by_name("q")
    time.sleep(1)
    search.send_keys(str(x))
    search.send_keys(Keys.RETURN)
    time.sleep(2)
    Site = driver.current_url
    driver.stop_client()
    driver.close()
    driver.quit()
    r = requests.get(Site)
    page = soup(r.text,'html.parser')
    #pegando informações dos mouses e seus preços
    info_mouses = page.find_all("div", {"class": "card card--prod"})
    name_mouse = []
    link_mouse = []
    mouse_preco = []

    name_mouse.clear()
    link_mouse.clear()
    mouse_preco.clear()


     
    if len(info_mouses)==0:
        info_mouses=page.find_all("div",{"class":"card card--offer card--cpc"})
        for x in info_mouses:
            link_mouse_find = x.find('a',{"class":"name"})
            nome_mouse = link_mouse_find.text
            link_mouse_real = 'https://www.zoom.com.br'+link_mouse_find['href']
            try:
                preco_em_real=x.find('span',{"class":"mainValue"}).text
            except:
                preco_em_real = "Valor Nao encontrado"
            preco_completo=preco_em_real
            name_mouse.append(nome_mouse)
            link_mouse.append(link_mouse_real)
            mouse_preco.append(preco_completo)
    else:
        for x in info_mouses:
            link_mouse_find = x.find('a',{"class":"name"})
            nome_mouse = link_mouse_find.text
            link_mouse_real = 'https://www.zoom.com.br'+link_mouse_find['href']
            try:
                preco_em_real=x.find('span',{"class":"mainValue"}).text
            except:
                preco_em_real = "Valor Nao encontrado"
            preco_completo=preco_em_real
            name_mouse.append(nome_mouse)
            link_mouse.append(link_mouse_real)
            mouse_preco.append(preco_completo)
                
        d = {'Nomes_Mouses':name_mouse,'Link':link_mouse,'Preço':mouse_preco}
        #transformo em Data Frame
        df = pd.DataFrame(d)
        #para printar linhas e colunas sem corte
        pd.set_option('display.max_colwidth', 1)
        pd.set_option('display.max_columns', None)
    return df

def mouse(bot, update):
    chat_id = update.message.chat_id
    bot.send_message(chat_id,"Hey Dude, i'm searching for your mouses !!")
    k = busca_zoom("mouse")
    for index, row in k.iterrows():
        bot.send_message(chat_id,str(row))

def keyboard(bot, update):
    chat_id = update.message.chat_id
    bot.send_message(chat_id,"Hey Dude, i'm searching for your keyboard !!")
    k = busca_zoom("teclado")
    for index, row in k.iterrows():
        bot.send_message(chat_id,str(row))
        
def gamer_chair(bot, update):
    chat_id = update.message.chat_id
    bot.send_message(chat_id,"Hey Dude, i'm searching for your gamer chair !!")
    k = busca_zoom("cadeira gamer")
    for index, row in k.iterrows():
        bot.send_message(chat_id,str(row))
        


def dog(bot, update):
    url = get_url()
    chat_id = update.message.chat_id
    bot.send_photo(chat_id,url)
    bot.send_message(chat_id,"Hey! here i find this dog for you, enjoy!!")
    bot.send_message(chat_id,"Hey if you want compare some mouse's prices, just write /mouse")
    bot.send_message(chat_id,"Hey if you want compare some Keyboard's prices, just write /keyboard") 
    bot.send_message(chat_id,"Hey if you want compare some gamer chair's prices, just write /gamer_chair")
    

    

def pichauMouse(bot,update):
    chat_id = update.message.chat_id 
    #data atual
    data_atual = date.today()
    
    #transformando data atual em texto no formato brasileiro
    data_em_texto = '{}-{}-{}'.format(data_atual.day, data_atual.month,data_atual.year)
    
    
    #SELENIUM PARA FAZER A BUSCA  o produto atualizar quantidade de produtos por pagina E RETORNAR URL
    driver = webdriver.Chrome()
    driver.get('https://www.pichau.com.br')
    time.sleep(2)
    search = driver.find_element_by_id("search")
    search.send_keys("Mouse Gamer")
    bot.send_message(chat_id,"Looking for Mouse Gamer")
    time.sleep(2)
    search.send_keys(Keys.RETURN)
    time.sleep(2)
    search = Select(driver.find_element_by_id('limiter'))
    time.sleep(2)
    search.select_by_value('48')
    time.sleep(3)
    Site = driver.current_url
    driver.stop_client()
    driver.close()
    driver.quit()
    bot.send_message(chat_id,"i have your mouses now i processing it, just a few seconfs")
    
    #usando BS4 para ler o HTML
    r = requests.get(Site)
    page = soup(r.text,'html.parser')
    
    #page_html = uClient.read()
    pegar_nome = page.findAll('a',{'class':'product-item-link'})
    pegar_preco_Parcelado = page.findAll('span',{'class':['price-installments']})
    bot.send_message(chat_id,"seconds 3...")
    preco_parcelado=[]
    Nome_Produto=[]
    Link_Produto=[]
    #len(pegar_preco)
    for i in range(0,35-1):
        #tratando o preco parcelado do produto
        preco_p = (str(pegar_preco_Parcelado[i]).replace('<span class="price-installments">','').split('</span>')[0].split('>')[1])
        preco_parcelado.append(preco_p)

        #tratando o nome do produto e o link

        link = str(pegar_nome[i]).split('"')[3]
        #print(str(pegar_nome[i]).split('"')[3])
        Link_Produto.append(link)
        product_name = str(pegar_nome[i]).split('"')[len(str(pegar_nome[i]).split('"'))-1].replace("</a>",'').replace(">",'').replace("\n",'')
        #print(str(pegar_nome[i]).split('"')[len(str(pegar_nome[i]).split('"'))-1].replace("</a>",'').replace(">",'').replace("\n",''))
        Nome_Produto.append(product_name)
    bot.send_message(chat_id,"seconds 2...")
    #transformando em dicionario os dados dos produtos
    d = {'preco_parcelado':preco_parcelado,'Nome_Produto':Nome_Produto,'Link_Produto':Link_Produto}
    
    #transformo em Data Frame
    df = pd.DataFrame(d)
    bot.send_message(chat_id,"seconds 1...")
    #para printar linhas e colunas sem corte
    pd.set_option('display.max_colwidth', 1)
    pd.set_option('display.max_columns', None)
    
    bot.send_message(chat_id,"saving file ...")
    #Export para uma pasta na area de trabalho
    export_csv = df.to_csv (r'C:\Users\dom.filho\Desktop\SM\MouseGamer_Pichau_Info_'+data_em_texto+'.csv', index = None, header=True)
    bot.send_message(chat_id,"ok, the file is saved in Desktop\SM\MouseGamer_Pichau_Info_ ")
    
    
    
def main():
    updater = Updater('969910557:AAHctzBMImkg4WaR2cIjwuNDtQe6pxb7r8w')
    dp = updater.dispatcher
    dp.add_handler(CommandHandler('dog',dog))
    dp.add_handler(CommandHandler('mouse',mouse))
    dp.add_handler(CommandHandler('keyboard',keyboard))
    dp.add_handler(CommandHandler('gamer_chair',gamer_chair))
    updater.start_polling()
    updater.idle()
    
if __name__ == '__main__':
    main()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:179: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details


In [8]:
def zoom (x):
    driver = webdriver.Chrome()
    driver.get('https://www.zoom.com.br/')
    time.sleep(2)
    driver.find_element_by_class_name("zsearch__fake").click()
    time.sleep(1)
    search = driver.find_element_by_name("q")
    time.sleep(1)
    search.send_keys(str(x))
    search.send_keys(Keys.RETURN)
    time.sleep(2)
    Site = driver.current_url
    driver.stop_client()
    driver.close()
    driver.quit()
    r = requests.get(Site)
    page = soup(r.text,'html.parser')
    #pegando informações dos mouses e seus preços
    info_mouses = page.find_all("div", {"class": "card card--prod"})
    name_mouse = []
    link_mouse = []
    mouse_preco = []

    name_mouse.clear()
    link_mouse.clear()
    mouse_preco.clear()


     
    if len(info_mouses)==0:
        info_mouses=page.find_all("div",{"class":"card card--offer card--cpc"})
        for x in info_mouses:
            link_mouse_find = x.find('a',{"class":"name"})
            nome_mouse = link_mouse_find.text
            link_mouse_real = 'https://www.zoom.com.br'+link_mouse_find['href']
            try:
                preco_em_real=x.find('span',{"class":"mainValue"}).text
                preco_completo=preco_em_real
            except:
                preco_em_real = "Valor Nao encontrado"
                preco_completo=preco_em_real
            name_mouse.append(nome_mouse)
            link_mouse.append(link_mouse_real)
            mouse_preco.append(preco_completo)
    else:
        for x in info_mouses:
            link_mouse_find = x.find('a',{"class":"name"})
            nome_mouse = link_mouse_find.text
            link_mouse_real = 'https://www.zoom.com.br'+link_mouse_find['href']
            try:
                preco_em_real=x.find('span',{"class":"mainValue"}).text
            except:
                preco_em_real = "Valor Nao encontrado"
            preco_completo=preco_em_real
            name_mouse.append(nome_mouse)
            link_mouse.append(link_mouse_real)
            mouse_preco.append(preco_completo)
                
        d = {'Nomes_Mouses':name_mouse,'Link':link_mouse,'Preço':mouse_preco}
        #transformo em Data Frame
        df = pd.DataFrame(d)
        #para printar linhas e colunas sem corte
        pd.set_option('display.max_colwidth', 1)
        pd.set_option('display.max_columns', None)
    return df

In [6]:
def busca_zoom(x):
    #SELENIUM PARA FAZER A BUSCA  o produto atualizar quantidade de produtos por pagina E RETORNAR URL
    driver = webdriver.Chrome()
    driver.get('https://www.zoom.com.br/')
    time.sleep(2)
    driver.find_element_by_class_name("zsearch__fake").click()
    time.sleep(1)
    search = driver.find_element_by_name("q")
    time.sleep(1)
    search.send_keys(str(x))
    search.send_keys(Keys.RETURN)
    time.sleep(2)
    Site = driver.current_url
    driver.stop_client()
    driver.close()
    driver.quit()
    r = requests.get(Site)
    page = soup(r.text,'html.parser')
    #pegando informações dos mouses e seus preços
    info_mouses = page.find_all("div", {"class": "card card--prod"})
    name_mouse = []
    link_mouse = []
    mouse_preco = []

    name_mouse.clear()
    link_mouse.clear()
    mouse_preco.clear()


     
    if len(info_mouses)==0:
        info_mouses=page.find_all("div",{"class":"card card--offer card--cpc"})
        for x in info_mouses:
            link_mouse_find = x.find('a',{"class":"name"})
            nome_mouse = link_mouse_find.text
            link_mouse_real = 'https://www.zoom.com.br'+link_mouse_find['href']
            try:
                preco_em_real=x.find('span',{"class":"mainValue"}).text
            except:
                preco_em_real = "Valor Nao encontrado"
            preco_completo=preco_em_real
            name_mouse.append(nome_mouse)
            link_mouse.append(link_mouse_real)
            mouse_preco.append(preco_completo)
    else:
        for x in info_mouses:
            link_mouse_find = x.find('a',{"class":"name"})
            nome_mouse = link_mouse_find.text
            link_mouse_real = 'https://www.zoom.com.br'+link_mouse_find['href']
            try:
                preco_em_real=x.find('span',{"class":"mainValue"}).text
            except:
                preco_em_real = "Valor Nao encontrado"
            preco_completo=preco_em_real
            name_mouse.append(nome_mouse)
            link_mouse.append(link_mouse_real)
            mouse_preco.append(preco_completo)
                
        d = {'Nomes_Mouses':name_mouse,'Link':link_mouse,'Preço':mouse_preco}
        #transformo em Data Frame
        df = pd.DataFrame(d)
        #para printar linhas e colunas sem corte
        pd.set_option('display.max_colwidth', 1)
        pd.set_option('display.max_columns', None)
    return df

In [7]:
k = busca_zoom("headset")

In [8]:
pd.DataFrame(k)

,Nomes_Mouses,Link,Preço
0,Headset com Microfone Knup KP-359,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-knup-kp-359?_lc=88&q=headset,R$ 59
1,Headset Gamer com Microfone HyperX Cloud Stinger Core PS4,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-gamer-com-microfone-hyperx-cloud-stinger-core?_lc=88&q=headset,R$ 269
2,Headset com Microfone C3 Tech Voicer Confort,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-com-microfone-filtro-para-ruidos-controle-de-volume-c3-tech-voicer-confort?_lc=88&q=headset,R$ 46
3,Headset com Microfone HyperX Cloud Stinger,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-hyperx-com-microfone-cloud-stinger?_lc=88&q=headset,R$ 442
4,Headset com Microfone Microsoft Lifechat LX-3000,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-com-microfone-controle-de-volume-microsoft-lifechat-lx-3000?_lc=88&q=headset,R$ 229
5,Headset com Microfone Corsair HS50 Stereo,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-corsair-com-microfone-hs50-stereo?_lc=88&q=headset,R$ 419
6,Headset com Microfone HyperX Cloud II,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-hyperx-cloud-ii?_lc=88&q=headset,R$ 599
7,Headset com Microfone HyperX Cloud Revolver,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-hyperx-com-microfone-cloud-revolver?_lc=88&q=headset,R$ 739
8,Headset com Microfone Philips SHM1900/00,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-com-microfone-controle-de-volume-philips-shm1900-00?_lc=88&q=headset,R$ 75
9,Headset com Microfone Logitech G230,https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-com-microfone-logitech-g230?_lc=88&q=headset,R$ 179


In [55]:
import pandas as pd


for index, row in k.iterrows():
    print(row)

Nomes_Mouses    Headset com Microfone Knup KP-359                                                    
Link            https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-knup-kp-359?_lc=88&q=headset
Preço           R$ 59,03                                                                             
Name: 0, dtype: object
Nomes_Mouses    Headset com Microfone C3 Tech Voicer Confort                                                                                                        
Link            https://www.zoom.com.br/fone-de-ouvido-e-headset/headset-com-microfone-filtro-para-ruidos-controle-de-volume-c3-tech-voicer-confort?_lc=88&q=headset
Preço           R$ 46,03                                                                                                                                            
Name: 1, dtype: object
Nomes_Mouses    Headset Gamer com Microfone HyperX Cloud Stinger Core PS4                                                              
Link            h

['apple', 'banana', 'cherry', 'orange']

[]